# World Bank: China GDP

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

## Read data: GDP growth (annual %) - China

In [5]:
# Japan, South Korea, Hong Kong (no Taiwan in DB?)
ne_asia_url = "https://comtrade.un.org/api/get?max=100000&type=C&freq=M&px=HS&ps=all&r=410%2C392%2C344&p=156&rg=2&cc=TOTAL&uitoken=2ca80db84aee68b93136acc75deb1487&fmt=csv"

# Philippines, Thailand, Indonedia, Singapore, Malaysia (no vietnam in DB?)
other_asia_url = "https://comtrade.un.org/api/get?max=100000&type=C&freq=M&px=HS&ps=all&r=360%2C458%2C608%2C702%2C764&p=156&rg=2&cc=TOTAL&uitoken=2ca80db84aee68b93136acc75deb1487&fmt=csv"

In [6]:
ne_src = pd.read_csv(ne_asia_url)
other_src = pd.read_csv(other_asia_url)

In [7]:
src = pd.concat([ne_src, other_src])

In [8]:
src.columns = src.columns.str.lower().str.replace(" ", "-")

In [9]:
src["month"] = pd.to_datetime(src["period"], format="%Y%m")

In [10]:
df = (
    src[
        [
            "year",
            "period",
            "period-desc.",
            "month",
            "trade-flow",
            "reporter",
            "partner",
            "trade-value-(us$)",
        ]
    ]
    .rename(columns={"trade-value-(us$)": "value"})
    .copy()
    .sort_values(["month", "reporter"])
    .reset_index(drop=True)
)

In [11]:
df.head()

,year,period,period-desc.,month,trade-flow,reporter,partner,value
0,2010,201001,January 2010,2010-01-01,Exports,"China, Hong Kong SAR",China,15190531668
1,2010,201001,January 2010,2010-01-01,Exports,Japan,China,10090748050
2,2010,201001,January 2010,2010-01-01,Exports,Malaysia,China,2102085022
3,2010,201001,January 2010,2010-01-01,Exports,Philippines,China,296687579
4,2010,201001,January 2010,2010-01-01,Exports,Singapore,China,4020124611


In [12]:
df["yychange"] = (df.groupby(["reporter"])["value"].pct_change(12)).round(2)

In [13]:
df = df[df["year"] > 2019]

In [14]:
((30023248204 - 19314806508) / 19314806508) * 100

55.441620352575995

In [22]:
df[df["reporter"].str.contains("Korea")]

,year,period,period-desc.,month,trade-flow,reporter,partner,value,yychange,region


In [16]:
regions = {
    "China, Hong Kong SAR": "Northeast Asia",
    "Japan": "Northeast Asia",
    "Rep. of Korea": "Northeast Asia",
    "Indonesia": "Other Asia",
    "Philippines": "Other Asia",
    "Singapore": "Other Asia",
    "Thailand": "Other Asia",
    "Malaysia": "Other Asia",
}

In [17]:
df["region"] = df["reporter"].map(regions)

In [18]:
# df["year"] = df["year"].astype(str)

In [20]:
(
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(
            "month:T",
            axis=alt.Axis(format="%m-%Y"),
            title=" ",
        ),
        y=alt.Y("yychange:Q", title=" ", axis=alt.Axis(format="%")),
        color=alt.condition(
            alt.datum.yychange > 0,
            alt.value("steelblue"),  # The positive color
            alt.value("orange"),  # The negative color
        ),
        facet=alt.Facet("reporter", title=" "),
    )
    .properties(width=200, height=200)
)

alt.Chart(...)